In [1]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance

In [7]:
client = QdrantClient(url="http://localhost:6333")
collection_name = "test_collection"

existing = [c.name for c in client.get_collections().collections]

if collection_name not in existing:
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=4, distance=Distance.DOT),
    )
    print(f"Created collection {collection_name!r}")
else:
    print(f"Collection {collection_name!r} already exists, skipping create.")


Collection 'test_collection' already exists, skipping create.


In [8]:
from qdrant_client.models import PointStruct

operation_info = client.upsert(
    collection_name="test_collection",
    wait=True,
    points=[
        PointStruct(id=1, vector=[0.05, 0.61, 0.76, 0.74], payload={"city": "Berlin"}),
        PointStruct(id=2, vector=[0.19, 0.81, 0.75, 0.11], payload={"city": "London"}),
        PointStruct(id=3, vector=[0.36, 0.55, 0.47, 0.94], payload={"city": "Moscow"}),
        PointStruct(
            id=4, vector=[0.18, 0.01, 0.85, 0.80], payload={"city": "New York"}
        ),
        PointStruct(id=5, vector=[0.24, 0.18, 0.22, 0.44], payload={"city": "Beijing"}),
        PointStruct(id=6, vector=[0.35, 0.08, 0.11, 0.44], payload={"city": "Mumbai"}),
    ],
)

print(operation_info)

operation_id=1 status=<UpdateStatus.COMPLETED: 'completed'>


In [10]:
search_result = client.query_points(
    collection_name="test_collection",
    query=[0.2, 0.1, 0.9, 0.7],
    with_payload=False,
    limit=3,
).points

print(search_result)

[ScoredPoint(id=4, version=1, score=1.362, payload=None, vector=None, shard_key=None, order_value=None), ScoredPoint(id=1, version=1, score=1.273, payload=None, vector=None, shard_key=None, order_value=None), ScoredPoint(id=3, version=1, score=1.208, payload=None, vector=None, shard_key=None, order_value=None)]


In [11]:
from qdrant_client.models import Filter, FieldCondition, MatchValue

search_result = client.query_points(
    collection_name="test_collection",
    query=[0.2, 0.1, 0.9, 0.7],
    query_filter=Filter(
        must=[FieldCondition(key="city", match=MatchValue(value="London"))]
    ),
    with_payload=True,
    limit=3,
).points

print(search_result)


[ScoredPoint(id=2, version=1, score=0.871, payload={'city': 'London'}, vector=None, shard_key=None, order_value=None)]
